In [3]:
from IPython.core.display import HTML
import pandas as pd 

pd.set_option('display.float_format', lambda x: '%.3f' % x)

def set_css_style(css_file_path):
    """
    Read the custom CSS file and load it into Jupyter.
    Pass the file path to the CSS file.
    """
    styles = open(css_file_path, "r").read()
    return HTML(styles)

set_css_style('styles/custom.css')

## Overview
- In this section, we will tackle some of the commands that allow us to manipulate `DataFrames` to either:

    - Merge multiple DataFrames. 

    - Concatenate (stack) data horizontally or vertically.


### Description of the Dataset used

- We will be using the spending data  (10k entries) used in the previous module, as well as a second file (medication_to_class.tsv) which contains information about the taxonomic class of each of the medication listed in the first table.

- The second table is tab delimited and contains the following information:
  
```python
medication_name                 opioid_drug     antibiotic_drug high_risk       antipsychotic_drug
ABACAVIR SULFATE                N               N               N                N
ABACAVIR SULFATE/LAMIVUDINE     N       		N       		N       		 N
ABACAVIR/DOLUTEGRAVIR/LAMIVUDI  N       		N       		N       		 N
ABACAVIR/LAMIVUDINE/ZIDOVUDINE  N       		N       		N       		 N
ABATACEPT          				N       		N       		N       		 N
```

Each line describes whether a drug is an opioid, an antibiotic, an antipsychotic or a high risk-drug (Y is YES, N is NO).


### Merging Series into a DataFrame 

- Merging, by default, joins the columns of two Series or DataFrames in a larger `DataFrame`.

- In its simplest form, merge occurs when joining multiple `Series` into a single `DataFrame`.




### Merging Series into a DataFrame 

![](images/merge_from_series.png)

- The underlying process is:
  - A new index comprised of both Series' indexes is created.
  - A value in a column is set to `NaN` if it's index did not exist in the corresponding `Series.`


### `pandas` `merge`

- `pandas`' has a method called `merge`, which uses a similar approch to merge `DataFrames`, except for the following two difference.


  - Rather than operate on the index, `merge` operates by default on `columns` with the same name across the `DataFrames`.
  
  
  - Indices which appear in only one of of the `DataFrame`s are discarded.
  
  


### `pandas` `merge` Example
- The Syntax for merging two dataframes, df1 and df2, is:

```python
    df1.merge(df2)
```


![](images/merge.png)


### Merging on Specific Columns

- By default, `pandas merges on columns that have the same label across both `DataFrames.`

- This default behavior can be modified by explicitly passing a column name (label) to the parameter `on=`.

- The column name passed to the parameter `on` needs to exist in both `DataFrames.`

![](images/merge_on.png)


### Merging on Specific Columns - Cont'd

- We can also specify column with differet labels by using the parameters `left_on=` and `right_on=`.

- `left_on` specifies the name of the column to use in the left `DataFrame`.
- `right_on` specifies the name of the column to use in the left `DataFrame`. 

- Explicitly specifying columns to merge on can avoid errors in large datasets with dozens of columns.


![](images/left_righ_on.png)





### Merging on Specific Columns - Cont'd

- It's also possible to merge on the index using `left_index=True` and/or `right_index=True`



- Merge can combin either `left_on` or `right_on`  with `left_index` and `right_index`



- When explicitly specifying column names to merge on, `merge` renames the columns having similar names and which are not used in the merge by appending suffix '_x' labels of the left table and _y to labels of the right table



# Merging Strategies

- By default, merge only retains values shared between both `DataFrame`s.
  - This is called an `inner` join strategy (see default parameters in merge method)



- This behavior can be modfied to give `merge` a different merging strategy



- There are four principal strategies for merging data. Those are:
    
    
| Merge strategy | Description |
|:--------|:-------------|
| `inner` | Keep keys that are present in both tables (intersection) |
| `left` | Keep keys found in left table and set the values in the right table to `NaN`  |
| `right` | Keep keys found in right table set values in the left table to `NaN` |
| `outer` | Keep the union of keys found in both right and left tables |
    
    

###  Merge With `inner` Strategy

![](images/inner.png)

###  Merge With `left` Strategy

![](images/left.png)

###  Merge With `right` Strategy

![](images/right.png)

###  Merge With `outer` Strategy
![](images/outer.png)

### Loading the Spending and the Medicine Class Data

- Combine the `DataFrame`s containing the spending data and the  medicine class data using `merge`.



- First, read both files and explore the columns names in both DataFrame

```python

spending_df = pd.read_table( 'data/spending_10k.csv', 
                                index_col="unique_id", 
                                dtype={"doctor_id":"object"}
                           )
med_class = pd.read_table("data/medication_to_class.tsv")

print(spending_df.columns)
print(med_class.columns)
```


In [4]:
spending_df = pd.read_table( 'data/spending_10k.tsv', 
                                index_col="unique_id", 
                                dtype={"doctor_id":"object"}
                           )


med_class = pd.read_table("data/medication_to_class.tsv")



print(spending_df.columns)
print("-" * 80)
print(med_class.columns)


Index(['doctor_id', 'specialty', 'medication', 'nb_beneficiaries', 'spending'], dtype='object')
--------------------------------------------------------------------------------
Index(['medication_name', 'opioid_drug', 'antibiotic_drug', 'high_risk',
       'antipsychotic_drug'],
      dtype='object')


# Merging the Spending and Medication Class

- Merge the `spending_df` and `med_class` `DataFrames` on the columns `medication` and `medication_name` respectively

- Since we don't want to show lines that have missing values, we will stick with the `inner` merge strategy.

```python
    spending_df.merge(med_class, left_on="medication", right_on="medication_name")
```

In [5]:
spending_df.merge(med_class, left_on="medication", right_on="medication_name").head()

,doctor_id,specialty,medication,nb_beneficiaries,spending,medication_name,opioid_drug,antibiotic_drug,high_risk,antipsychotic_drug
0,1255626040,FAMILY PRACTICE,METFORMIN HCL,30,135.240,METFORMIN HCL,N,N,N,N
1,1912118928,FAMILY PRACTICE,METFORMIN HCL,20,1066.900,METFORMIN HCL,N,N,N,N
2,1023116894,FAMILY PRACTICE,METFORMIN HCL,161,726.600,METFORMIN HCL,N,N,N,N
3,1871916031,FAMILY PRACTICE,METFORMIN HCL,59,235.410,METFORMIN HCL,N,N,N,N
4,1467565440,FAMILY PRACTICE,METFORMIN HCL,22,3373.470,METFORMIN HCL,N,N,N,N


### Stacking Data Along an Axis

- Combining data consists of stacking it along an `axis.`

- This is also known as `binding` or `stacking`.

- This enables joining multiple `DataFrames` into a single larger one

![](images/stacking.png)



# Stacking with `pandas` `concat`

- Concatenation in Python is done using the `concat` `pandas` `function.`

  - Note that this is a `function`,  not a `DataFrame` object's method. 
  - `concat` is executed as:



# Stacking with `pandas` `concat` Example

```python
pd.concat([s1, s2, s3])
``` 

```python
# INPUT
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])

pd.concat([s1, s2, s3])
```

```
a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64
```


### Stacking DataFrames as Rows

- By default, concatenation uses `axis=0` (rows) 

- When stacking as rows the `DataFrame`s are matched on their column labels
- Values of column that do not exist in a `DataFrame` are replace by `NaN`
![](images/stacking_row_ex1.png)

### Stacking DataFrames as Columns

- The change of behavior required to stack `DataFrame`s as columns needs to be explicitely specified using `axis=rows` (`axis=1`) 

- When stacking as columns the `DataFrame`s are match on their rows labels
- Values of rows that do not exist in a `DataFrame` are replace by `NaN`
![](images/stacking_cols_ex1.png)

### PRACTICAL

- Start with a new Jupyter Notebook.

- Read the file `data/joining_data_practical.tsv` located in the data folder into a new pandas DataFrame called `spending_practical_df`.

  - Make sure you import the appropriate module first.

- Read the file `data/medication_to_class_part_1.tsv` located in the data folder into a new pandas DataFrame called `medication_to_class_p1`.


1- Use the appropriate type of merging strategy (inner, outer, left or right) to identify medicines in spending_practical_df for which we do not have a class, i.e., no entry in the `medication_to_class_p1` DataFrame. 

   - Hint 2, which merging strategy will create NaN for medicine that does not have an associated class entry? Count the number of NaN yields the number of medicines missing from  `medication_to_class_p1`.
   - Hint 2, which column names will you merge on. Use `head` to preview which columns you need to merge on. 
 

2- We have a second file called `medication_to_class_2.tsv` which contains additional medication classes 
 - Read the file `medication_to_class_2.tsv` into a new DataFrame called `medication_to_class_p1` and concat both `medication_to_class_p1` and `medication_to_class_p2` into a dataframe called medication_to_class_ALL
 - Repeat question 1. using medication_to_class_ALL. How many medicines are still missing a class?

 - What are the names of the medicines still missing? 

